In [2]:
from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder
import torch        
from data_processing.sequences.sequencing import get_pairs

In [3]:
get_pairs("ch07014")

'ch014'

In [20]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device


## Autoencoder

In [21]:
### Initialization of the Autoencoder 
SEQ_LEN = 3000
HIDDEN_DIM = 512
ENCODING_SIZE = 64
model = AutoEncoder(vocab_size=100, embedding_size=HIDDEN_DIM, encoding_size=ENCODING_SIZE, sequence_len=SEQ_LEN)


/Users/carlomarx/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [22]:
#let's assume we have a batch of 2 people
x = torch.randint(1,99, size=(2,SEQ_LEN))
y = model(x) 
## returns the original shape


In [23]:
### only to use the encoder part 
y = model.encode(x) # here y contains embedding of a survey per row

In [24]:
y

tensor([[1.0832, 0.9053, 1.0793, 1.0069, 1.0820, 1.0080, 1.0972, 1.1351, 1.1825,
         1.1825, 1.1798, 1.2497, 1.0592, 1.1617, 1.0967, 0.9440, 0.9736, 1.0652,
         1.1814, 1.1295, 1.1983, 0.9670, 1.0642, 0.9383, 1.0616, 1.2068, 1.0834,
         1.1106, 1.0930, 1.1473, 1.0343, 1.2510, 1.1105, 1.1286, 1.0923, 1.0668,
         1.0937, 1.0260, 1.0193, 1.0188, 1.0924, 1.0847, 1.0894, 1.1251, 1.1681,
         1.0378, 1.0206, 1.2475, 1.1310, 0.9895, 1.1751, 0.9599, 1.1929, 1.1732,
         1.1567, 0.9172, 0.8553, 1.0445, 1.0713, 1.0777, 1.0420, 1.1589, 0.9551,
         1.1668],
        [1.1318, 1.0795, 1.1133, 1.0535, 1.2317, 1.0746, 1.0192, 1.1769, 1.0959,
         1.1960, 1.0426, 0.9671, 0.9651, 1.0878, 1.1340, 1.1135, 1.1042, 1.0427,
         1.1906, 1.0300, 0.9913, 1.0032, 1.0898, 1.0759, 1.0755, 1.0717, 1.1187,
         1.1150, 1.0206, 1.0369, 1.0803, 1.0738, 0.9569, 1.0141, 1.1088, 1.0496,
         1.0675, 1.1581, 1.1693, 1.1407, 1.0574, 1.0215, 0.9724, 1.0124, 0.9426,
         1

In [38]:
multihead_attn = torch.nn.MultiheadAttention(HIDDEN_DIM, 8, vdim=16, kdim=16)
_x = model.embedding(x)
_k, _v = torch.rand((2,SEQ_LEN,16)), torch.rand((2,SEQ_LEN,16))
_x , _ = multihead_attn(_x, _k,_v, need_weights=False)
_x.shape

torch.Size([2, 3000, 512])

## RNN

In [8]:
## 
# input_size -> the size of the embedding of the autoencoder model
# hidden_size -> the size of the RNN to use in the decoder (the input_size and hidden_size can be different)
model = GRUDecoder(input_size=6, hidden_size=10, max_seq_len=4).to(get_device())
loss_f = nn.BCEWithLogitsLoss()
solver = torch.optim.AdamW(model.parameters())

The model is going to set all input MASK to None
Using MPS (Metal Performance Shaders) device


/Users/carlomarx/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/carlomarx/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# This is just an example

MAX_SEQ_LEN = 4 # max number of surveyas a person (in our dataset can have)
INPUT_SIZE = 6 # hidden dimmensions of autoencodder.

# let's say we have a person who only have 3 surveys
x0 = torch.rand(INPUT_SIZE) # embedding for the 1st survey 
x1 = torch.rand(INPUT_SIZE) # embedding for the 2nd survey
x2 = torch.rand(INPUT_SIZE)
x3 = torch.rand(INPUT_SIZE)
# the tensor for the person should be on the shape [MAX_SEQ_LEN, INPUT_SIZE]

e = torch.zeros(MAX_SEQ_LEN, INPUT_SIZE)
e[0] = x0
e[1] = x1
e[2] = x2
e = e.to(get_device()) # so this is a tensor for the person
#we also need to specify that the sequence has 'empty' embeddings
#mask = torch.BoolTensor([True, True, False, False]).to(get_device()) # the last two dimensions are empty
## it is important that you append existing survey embeddings right next to each other (even if the year is missign between them, they should be still appended one after another)

## let assume we have a batch of people, I am reusing the same person, but in the pipeline is should be different people
# the batch size is 3 here 

x = torch.stack([e,e,e])
y = torch.tensor([1.,1,1.]).to(get_device())
#mask = torch.stack([mask, mask, mask])

Using MPS (Metal Performance Shaders) device
Using MPS (Metal Performance Shaders) device


In [12]:
for i in range(100):
    solver.zero_grad()
    #xx = torch.nn.functional.sigmoid(model(x, mask))
    xx = model(x, None)

    loss = loss_f(xx.view(-1), y.view(-1))
    loss.backward()
    solver.step()

In [13]:
loss

tensor(0.0794, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)